In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
import pickle
import datetime
import time
import re

In [2]:
df = pd.read_parquet('merge_book_2015_20.parquet', engine='pyarrow')

In [3]:
df.shape

(1773701, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773701 entries, 0 to 1773700
Data columns (total 14 columns):
 #   Column       Dtype  
---  ------       -----  
 0   reviewerID   object 
 1   asin         object 
 2   overall      int16  
 3   title        object 
 4   verified     bool   
 5   summary      object 
 6   reviewText   object 
 7   timestamp    object 
 8   description  object 
 9   price        float64
 10  Format:      object 
 11  category     object 
 12  main_cat     object 
 13  brand        object 
dtypes: bool(1), float64(1), int16(1), object(11)
memory usage: 167.5+ MB


In [5]:
df.head()

,reviewerID,asin,overall,title,verified,summary,reviewText,timestamp,description,price,Format:,category,main_cat,brand
0,AVP0HXC9FG790,0001713353,5,"King, the Mice and the Cheese (Beginner Books)",True,Five Stars,The kids loved it!,2016-06-20,"THE KING, THE MICE AND THE CHEESE follows a fo...",12.33,None,"[Books, Children's Books, Literature & Fiction]",Books,Visit Amazon's Nancy Gurney Page
1,A2RE7WG349NV5D,0001713353,5,"King, the Mice and the Cheese (Beginner Books)",False,Five Stars,LOVE IT,2015-07-09,"THE KING, THE MICE AND THE CHEESE follows a fo...",12.33,Paperback,"[Books, Children's Books, Literature & Fiction]",Books,Visit Amazon's Nancy Gurney Page
2,A2CVLIZ9ELU88,0001061240,5,The golden treasury of poetry,True,A wonderful poetry collection with superior co...,I had this book as a child and loved it. The c...,2016-10-06,Collection of Poetry,23.98,Hardcover,"[Books, Children's Books, Literature & Fiction]",Books,Louis (ed) UNTERMEYER
3,A3LAQSHW9IVKUO,0002005263,5,The Sinister Pig,True,You will want to take notes,"Typical Hillerman, the only mystery set in mod...",2016-11-06,LARGE PRINT HARDBACK.,5.85,Mass Market Paperback,None,Books,Visit Amazon's Tony. Hillerman Page
4,A2B4R7L8FX9BGT,0002005263,5,The Sinister Pig,True,Five Stars,Love it.,2016-01-09,LARGE PRINT HARDBACK.,5.85,Kindle Edition,None,Books,Visit Amazon's Tony. Hillerman Page


# unixReviewTime 복원 - model 돌릴 때, unixtime으로 필요해서

In [6]:
df['unixReviewTime'] = df['timestamp'].apply(lambda x : time.mktime(x.timetuple()))

# category 리스트 없애기 및 장르 추출 (meta_book 원본 데이터에서 전처리)

In [7]:
mb = pd.read_csv('meta_book_original.csv')

### genre 추출 위한 탐색과정

In [90]:
category_list = []
for i in tqdm(mb['category']):
    if not pd.isnull(i):
        i = re.sub(r"[^a-z A-Z0-9]", "", i).lower()
        for j in i.split():
            category_list.append(j)
        # i = eval(i)
        # for j in i:
        #     if j != 'Books':
        #         for k in j.split(';'):
        #             for p in k.split(','):
        #                 category_list.append(p.lower().replace('&','').replace('amp','').strip())

100%|██████████| 2934949/2934949 [00:15<00:00, 185876.78it/s]


In [94]:
category_words = []
for i in sorted(dict(Counter(category_list)).items(),key=lambda x : x[1],reverse=True):
    category_words.append(i[0])

In [99]:
category_list2 = []
for i in tqdm(mb['category']):
    if not pd.isnull(i):
        i = eval(i)
        for j in i:
            if j != 'Books':
                for k in j.split(';'):
                    for p in k.split(','):
                        category_list2.append(p.lower().replace('&','').replace('amp','').strip())

100%|██████████| 2934949/2934949 [00:35<00:00, 82260.54it/s] 


In [163]:
# len(sorted(dict(Counter(category_list)).items(),key=lambda x : x[1],reverse=True))

### 핵심 단어 추출

In [8]:
category_genre_dic = {'history' : 'history', 'science fiction' : 'sf','child' : 'child','fiction' : 'fiction','novel':'fiction',
                      'arts': 'arts','design': 'arts','painting': 'arts','drawing': 'arts','photography': 'arts','poetry' : 'poetry',
                      'mystery' : 'mystery', 'bible': 'religion','christ': 'religion','spirituality': 'religion','religion': 'religion',
                      'catholic': 'religion','church': 'religion','worship': 'religion','myth': 'religion','buddhism': 'religion',
                      'hobb' : 'hobby','travel' : 'hobby','home' : 'hobby','fishing' : 'hobby','hunting' : 'hobby', 'social science': 'social',
                      'sociology': 'social','fantasy' : 'fantasy','memoir' : 'memoir','biograph' : 'memoir','suspense' : 'suspense',
                      'politic' : 'politic','government' : 'politic','business' : 'business','money' : 'business','america':'america',
                      'united state':'america','music' : 'music','engineering' : 'technology','technology' : 'technology',
                      'computer' : 'technology','software' : 'technology','system' : 'technology','hardware' : 'technology',
                      'system' : 'technology','humor' : 'humor','comic' : 'humor','romance' : 'romance','medic' : 'science',
                      'astronomy' : 'science','chemistry' : 'science','psychology' : 'science','humanit' : 'humanity','drama' : 'humanity',
                      'self-help' : 'self-help','thriller' : 'thriller', 'coek' : 'cook','wine' : 'cook','food' : 'cook', 
                      'dieting' : 'cook', 'baking' : 'cook', 'health' : 'health','fitness' : 'health','outdoor' : 'health',
                      'educat' : 'education','math' : 'education','school' : 'education','foreign language' : 'education',
                      'entertainment' : 'entertainment','criticism' : 'criticism','animal' : 'animal','pets' : 'animal',
                      'sports' : 'sports','action' : 'action','adventure' : 'action','science':'science',
                     'grammar' : 'education', 'law' : 'law'}

### 새로운 장르 컬럼 생성

In [165]:
category_dic = {}
for n, i in enumerate(tqdm(mb['category'])):
    if pd.isnull(i):
        category_dic[n] = ['etc']
        continue
    one_category_list = []
    i = eval(i)
    for j in i:
        j = j.lower()
        for g in category_genre_dic:
            if g in j:
                one_category_list.append(category_genre_dic[g])
                break
    if not one_category_list:
        category_dic[n] = ['etc']
        continue
    category_dic[n] = one_category_list

100%|██████████| 2934949/2934949 [01:08<00:00, 43146.18it/s] 


In [166]:
mb['category2'] = category_dic.values()

In [167]:
a = []
for i in tqdm(range(len(mb))):
    if not pd.isnull(mb['category'].iloc[i]) and mb['category2'].iloc[i][0] == 'etc':
        a.append(i)

100%|██████████| 2934949/2934949 [00:50<00:00, 58467.28it/s]


In [173]:
# 전체 293만 중 8만 정도 etc 됨
print(len(mb.iloc[a]))
mb.iloc[a]['category'].value_counts()[:5]

84546


['Books', 'Reference', 'Writing, Research &amp; Publishing Guides']    13315
['Books', 'Reference']                                                 11644
['Books', 'Reference', 'Writing, Research & Publishing Guides']         7092
['Books', 'Parenting &amp; Relationships', 'Family Relationships']      4389
['Books', 'New, Used &amp; Rental Textbooks']                           4383
Name: category, dtype: int64

# df에 장르 컬럼으로 추가

In [9]:
category_dic = {}
for n, i in enumerate(tqdm(df['category'])):
    if type(i) != np.ndarray:
        category_dic[n] = ['etc']
        continue
    one_category_list = []
    # i = eval(i)
    for j in i:
        j = j.lower()
        for g in category_genre_dic:
            if g in j:
                one_category_list.append(category_genre_dic[g])
                break
    if not one_category_list:
        category_dic[n] = ['etc']
        continue
    category_dic[n] = list(set(one_category_list))

100%|██████████| 1773701/1773701 [00:31<00:00, 55608.72it/s]


In [10]:
df['genre'] = category_dic.values()

In [11]:
a = []
for i in tqdm(range(len(df))):
    if type(df['category'].iloc[i]) == np.ndarray and df['genre'].iloc[i][0] == 'etc':
        a.append(i)
# 0.8%만 장르 etc (none값은 제외하고)
len(a) / len(df) * 100

100%|██████████| 1773701/1773701 [00:31<00:00, 55684.10it/s]


In [26]:
df.drop('category',axis=1,inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773701 entries, 0 to 1773700
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   reviewerID      object 
 1   asin            object 
 2   overall         int16  
 3   title           object 
 4   verified        bool   
 5   summary         object 
 6   reviewText      object 
 7   timestamp       object 
 8   description     object 
 9   price           float64
 10  Format:         object 
 11  main_cat        object 
 12  brand           object 
 13  unixReviewTime  float64
 14  genre           object 
dtypes: bool(1), float64(2), int16(1), object(11)
memory usage: 181.0+ MB


In [31]:
df.to_parquet('merge_book_2015_20_final.parquet', engine='pyarrow', index=False)

### 장르 순위

In [37]:
a = []
for i in tqdm(df['genre']):
    a.append(str(i))

100%|██████████| 1773701/1773701 [00:01<00:00, 1082165.07it/s]


In [40]:
Counter(a)

Counter({"['child', 'fiction']": 13812,
         "['etc']": 77868,
         "['fiction']": 335979,
         "['sf']": 63755,
         "['religion']": 56662,
         "['poetry', 'fiction']": 4951,
         "['hobby']": 34480,
         "['child', 'animal']": 8736,
         "['history']": 15974,
         "['child']": 45691,
         "['humor']": 16542,
         "['child', 'history']": 1968,
         "['self-help']": 15533,
         "['religion', 'fiction']": 46404,
         "['suspense', 'mystery']": 106662,
         "['memoir']": 25254,
         "['business']": 20297,
         "['action', 'fiction']": 31692,
         "['suspense', 'romance']": 44789,
         "['child', 'science']": 3005,
         "['health']": 2107,
         "['social']": 10049,
         "['humanity']": 16426,
         "['fantasy', 'sf']": 55397,
         "['cook']": 33340,
         "['humanity', 'fiction']": 3065,
         "['mystery']": 77667,
         "['america', 'fiction']": 50106,
         "['education', 'science

In [41]:
df

,reviewerID,asin,overall,title,verified,summary,reviewText,timestamp,description,price,Format:,main_cat,brand,unixReviewTime,genre
0,AVP0HXC9FG790,0001713353,5,"King, the Mice and the Cheese (Beginner Books)",True,Five Stars,The kids loved it!,2016-06-20,"THE KING, THE MICE AND THE CHEESE follows a fo...",12.33,None,Books,Visit Amazon's Nancy Gurney Page,1.466381e+09,"[child, fiction]"
1,A2RE7WG349NV5D,0001713353,5,"King, the Mice and the Cheese (Beginner Books)",False,Five Stars,LOVE IT,2015-07-09,"THE KING, THE MICE AND THE CHEESE follows a fo...",12.33,Paperback,Books,Visit Amazon's Nancy Gurney Page,1.436400e+09,"[child, fiction]"
2,A2CVLIZ9ELU88,0001061240,5,The golden treasury of poetry,True,A wonderful poetry collection with superior co...,I had this book as a child and loved it. The c...,2016-10-06,Collection of Poetry,23.98,Hardcover,Books,Louis (ed) UNTERMEYER,1.475712e+09,"[child, fiction]"
3,A3LAQSHW9IVKUO,0002005263,5,The Sinister Pig,True,You will want to take notes,"Typical Hillerman, the only mystery set in mod...",2016-11-06,LARGE PRINT HARDBACK.,5.85,Mass Market Paperback,Books,Visit Amazon's Tony. Hillerman Page,1.478390e+09,[etc]
4,A2B4R7L8FX9BGT,0002005263,5,The Sinister Pig,True,Five Stars,Love it.,2016-01-09,LARGE PRINT HARDBACK.,5.85,Kindle Edition,Books,Visit Amazon's Tony. Hillerman Page,1.452298e+09,[etc]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773696,A36OTZP3JUZSYB,B01HJ56VC4,4,The Seance (Jinn Series Book 1) eBook,True,Very Nicely Done!,I really enjoyed this short story and Evelyn. ...,2017-06-24,None,NaN,Kindle Edition,Buy a Kindle,Visit Amazon's Ayse Hafiza Page,1.498262e+09,[fiction]
1773697,A3KQ9BFVH82015,B01HJ56VC4,5,The Seance (Jinn Series Book 1) eBook,False,"I loved ""Seance"" by Ayse Hafiza","I loved ""Seance"" by Ayse Hafiza, it reminded m...",2017-06-17,None,NaN,Kindle Edition,Buy a Kindle,Visit Amazon's Ayse Hafiza Page,1.497658e+09,[fiction]
1773698,AX9E9GSUYYPKS,B01HJ2RQGM,5,PLAYBOY MAGAZINE JULY/AUGUST 2016,True,Five Stars,No problems with seller or item,2016-07-17,"this product was never view, only taken out of...",11.75,None,Books,None,1.468714e+09,[fiction]
1773699,A28Q75F97EPZ2A,B01HJ6NAUY,3,The Lusca eBook,False,Squid,Not a bad story. Giant squid and it's eels com...,2017-08-05,None,NaN,Kindle Edition,Buy a Kindle,Lisa R. Hazard,1.501891e+09,[fiction]
